# Лабораторная работа №2 по курсу "Машинное обучение"

Тояков Артем М8О-407Б-18 Вариант:22

## Датасет

Для начала я загружу датасет пассажиров с титаника из первой лабораторной работы.

In [75]:
import pandas as pd
import numpy as np

data = pd.read_csv('titanic.csv')
data = data.drop(['PassengerId', 'Ticket', 'Name', 'Cabin'], axis=1)
data['Sex'] = data['Sex'].map({'female': 1, 'male': 0}).astype(int)
data['Age'] = data['Age'].fillna(data['Age'].mean())
freq_port = data.Embarked.dropna().mode()[0]
data['Embarked'] = data['Embarked'].fillna(freq_port)
data['Embarked'] = data['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)
data

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,0,22.000000,1,0,7.2500,0
1,1,1,1,38.000000,1,0,71.2833,1
2,1,3,1,26.000000,0,0,7.9250,0
3,1,1,1,35.000000,1,0,53.1000,0
4,0,3,0,35.000000,0,0,8.0500,0
...,...,...,...,...,...,...,...,...
886,0,2,0,27.000000,0,0,13.0000,0
887,1,1,1,19.000000,0,0,30.0000,0
888,0,3,1,29.699118,1,2,23.4500,0
889,1,1,0,26.000000,0,0,30.0000,1


Наша задача определить: выжил пассажир или нет.

In [76]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import *

X, y = data.drop(columns=['Survived']).to_numpy(), np.array(data['Survived'])
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.15, random_state=15)

## Логистическая регрессия

Добавим удобные функции для оценки результата и сигмоиду, которая будет использоваться в логистической регрессии.

In [77]:
def score_prediction(y_pred, y_test):
    print("confusion matrix:\n", confusion_matrix(y_test, y_pred))
    print("accuracy:", accuracy_score(y_test, y_pred))
    print("precision:", precision_score(y_test, y_pred))
    print("recall:", recall_score(y_test, y_pred))
    print("f1_score", f1_score(y_test, y_pred))
    
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

Реализацию логистической регрессии запишем в класс.

In [78]:
class MyLogisticRegression:
    def __init__(self, lr, max_iter = 10000):
        self.max_iter = max_iter
        self.lr = lr

    def fit(self, X, y):
        X = np.hstack((np.full((X.shape[0], 1), 1),X))
        weights = np.random.rand(X.shape[1])
        for i in range(self.max_iter):
            old_weights = weights.copy()
            weights -= self.lr * np.dot(X.T, sigmoid(np.dot(X, weights)) - y) / len(X)
        self.weights = weights

    def predict(self, X):
        X = np.hstack((np.full((X.shape[0], 1), 1), X))
        labels = []
        for row in X:
            labels.append(int(sigmoid(np.dot(row, self.weights)) > 0.5))
        return labels

Проверим, насколько точна реализованная нами регрессия.

In [79]:
from sklearn.preprocessing import *

max_accuracy = 0
max_params = (0,0)
y_pred_max = 0
for lr, max_iter in [(0.1, 100), (0.01, 100), (0.1, 1000), (0.01, 1000), (0.1, 10000), (0.01, 10000)]:
    model = MyLogisticRegression(lr=lr, max_iter=max_iter)
    model.fit(Xtrain, Ytrain)
    y_pred = model.predict(Xtest)
    acc = accuracy_score(y_pred, Ytest)
    if acc > max_accuracy:
        max_params = (lr, max_iter)
        max_accuracy = acc
        y_pred_max = y_pred
print("My implementation: max accuracy with learning rate = {} and max iterations = {}"
      .format(*max_params))
score_prediction(y_pred_max, y_test)

My implementation: max accuracy with learning rate = 0.01 and max iterations = 1000
confusion matrix:
 [[48 26]
 [ 8 52]]
accuracy: 0.746268656716418
precision: 0.6666666666666666
recall: 0.8666666666666667
f1_score 0.7536231884057971


Посмотрим теперь на точность регрессии из библиотеки sklearn.

In [80]:
model = LogisticRegression(max_iter=10000)
model.fit(Xtrain,Ytrain)
print("Sklearn implementation accuracy {}".format(accuracy_score(model.predict(Xtest), Ytest)))

Sklearn implementation accuracy 0.7910447761194029


Как можно заметить, точности почти идентичны. Точность моей реализации не идельна, но при этом довольно высокая, из чего можно сделать вывод, что логистическая регрессия хорошо подходит для решения данной задачи.

## Дерево решений

Вершину дерева и её параметры поместим в класс.

In [81]:
class Node:
    def __init__(self, index, value):
        self.index = index
        self.value = value
        self.left = None
        self.right = None

Используя алгоритм ID3 построим дерево решений.

In [87]:
def Gini(X, Y, i, v):
    YL = Y[X[::, i] <= v]
    YR = Y[X[::, i] > v]
    
    yl1 = YL.sum()
    yl0 = YL.shape[0] - yl1
    
    yr1 = YR.sum()
    yr0 = YR.shape[0] - yr1
    
    return (yl0 * (yl0 + 1) + yl1 * (yl1 + 1) + yr0 * (yr0 + 1) + yr1 * (yr1 + 1))

def BestGini(X, Y):
    index = 0
    value = 0
    gini = 0
    
    for i in range(X.shape[1]):
        for v in set(X[::, i]):
            g = Gini(X, Y, i, v)
            
            if g < gini or gini == 0:
                gini = g
                index = i
                value = v
    
    return index, value

def ID3(X, Y, depth):
    #все вершины одного класса
    if Y.min() == Y.max():
        return Node(-1, Y[0])
    if depth == 0:
        return Node(-1, round(Y.mean()))
    
    Node(-1, round(Y.mean()))
    #используем критерий Джини
    index, value = BestGini(X, Y)
    
    XL = X[X[::, index] <= value]
    XR = X[X[::, index] > value]
    YL = Y[X[::, index] <= value]
    YR = Y[X[::, index] > value]
    
    if XL.shape[0] == 0 or XR.shape[0] == 0:
        return Node(-1, round(Y.mean()))
    
    #создаём вершину и её поддеревья
    n = Node(index, value)
    n.left = ID3(XL, YL, depth - 1)
    n.right = ID3(XR, YR, depth - 1)
    return n

#вычислить значение по дереву
def getClass(tree, x):
    if tree.index == -1:
        return tree.value
    
    if x[tree.index] <= tree.value:
        return getClass(tree.left, x)
    else:
        return getClass(tree.right, x)

Определим ещё одну функцию для оценки точности алгоритма.

In [89]:
def print_accuracy(y, y0):
    tp, fp, tn, fn = 0, 0, 0, 0
    
    for i in range(y.shape[0]):
        if y[i] == 0 and y0[i] == 0: tn += 1
        if y[i] == 1 and y0[i] == 0: fn += 1
        if y[i] == 0 and y0[i] == 1: fp += 1
        if y[i] == 1 and y0[i] == 1: tp += 1
        
    print('Accuracy:', (tp + tn) / (tp + tn + fp + fn))
    print('Precision:', tp / (tp + fp))
    print('Recall:', tp / (tp + fn))

In [90]:
tree = ID3(X_train, y_train, 15)

tree_y0 = np.zeros(y_test.shape[0])

for i in range(y_test.shape[0]):
    tree_y0[i] = getClass(tree, X_test[i])

print_accuracy(y_test, tree_y0)

Accuracy: 0.6940298507462687
Precision: 0.7714285714285715
Recall: 0.45


In [91]:
from sklearn.tree import DecisionTreeClassifier
sk_tree = DecisionTreeClassifier()
sk_tree.fit(X_train, y_train)

print_accuracy(y_test, sk_tree.predict(Xtest))

Accuracy: 0.7761194029850746
Precision: 0.7272727272727273
Recall: 0.8


Как можно заметить, sklearn гораздо точнее решил задачу, чем написанный нами алгоритм.

## SVM

In [93]:
svm_w = np.zeros(Xtrain.shape[1])
svm_b = 0

def f(x):
    return 1 if np.dot(svm_w, x) - svm_b > 0 else -1

#преобразуем Ytrain из [0, 1] в [-1, 1]
svm_y = 2 * Ytrain - 1

#обучение
rate = 0.01
lambd = 0.02

for repeat in range(250):
    for i in range(Xtrain.shape[0]):
        x = Xtrain[i]
        if 1 - svm_y[i] * (np.dot(svm_w, x) - svm_b) >= 0:
            svm_w = svm_w * (1 - rate * lambd) + rate * svm_y[i] * x
            svm_b = svm_b - rate * svm_y[i]
        else:
            svm_w = svm_w * (1 - rate * lambd)

In [101]:
svm_y0 = np.zeros(y_test.shape[0])

for i in range(y_test.shape[0]):
    if f(X_test[i]) < 0:
        svm_y0[i] = 0
    else:
        svm_y0[i] = 1

print_accuracy(y_test, svm_y0)

Accuracy: 0.6940298507462687
Precision: 0.6862745098039216
Recall: 0.5833333333333334


In [100]:
from sklearn.svm import SVC
sk_svm = SVC()
sk_svm.fit(X_train, y_train)

print_accuracy(y_test, sk_svm.predict(X_test))

Accuracy: 0.6791044776119403
Precision: 0.8148148148148148
Recall: 0.36666666666666664


На удивление, sklearn показал такую же точность как и наш алгоритм, но параметры precision и recall гораздо меньше именно в реализации sklearn. Возможно это из-за того что у нас не так много данных, и на бОльших объёмах алгоритм покажет более высокую эффективность. 